<a href="https://colab.research.google.com/github/Ajjme/Feedback_Systems/blob/main/NWS_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#libraries
import requests
from requests.exceptions import HTTPError, Timeout, RequestException
import pandas as pd

In [9]:
#Request Function
def make_nws_request(endpoint, user_agent):
    headers = {
        "User-Agent": user_agent,
        "Accept": "application/geo+json"  # recommended by NWS Geo Jsons are easier to map

    }

    try:
        response = requests.get(
                       endpoint,
                       headers=headers
                   )
        # Raise HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status()
        return response.json()

    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status code: {response.status_code}")
    except Timeout as timeout_err:
        print(f"Request timed out: {timeout_err}")
    except RequestException as req_err:
        print(f"Request error: {req_err}")

    return None  # Return None if an error occurred

In [10]:
if __name__ == "__main__":
    # Sample user agent
    user_agent = "Weatherdata_collector/1.0" # you can name this whatever you want
    BASE_URL = "https://api.weather.gov"

    # Get forecast for a specific location - Durham
    lat, lon = 36.0, -79.0

    #Put Params into a url String to acess API
    forecast_url = f"{BASE_URL}/points/{lat},{lon}"

    #run above function
    data = make_nws_request(forecast_url, user_agent)

    if data:
        print(data)
    else:
        print("Failed to retrieve data.")

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld', {'@version': '1.1', 'wx': 'https://api.weather.gov/ontology#', 's': 'https://schema.org/', 'geo': 'http://www.opengis.net/ont/geosparql#', 'unit': 'http://codes.wmo.int/common/unit/', '@vocab': 'https://api.weather.gov/ontology#', 'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'}, 'city': 's:addressLocality', 'state': 's:addressRegion', 'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'}, 'bearing': {'@type': 's:QuantitativeValue'}, 'value': {'@id': 's:value'}, 'unitCode': {'@id': 's:unitCode', '@type': '@id'}, 'forecastOffice': {'@type': '@id'}, 'forecastGridData': {'@type': '@id'}, 'publicZone': {'@type': '@id'}, 'county': {'@type': '@id'}}], 'id': 'https://api.weather.gov/points/36,-79', 'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-79, 36]}, 'properties': {'@id': 'https://api.weather.gov/points/36,-79', '@type': 'wx:Point', 'cwa': 'RAH', 'forecastOffice': 'https

This shows us all the options from NWS for the point that we entered.

In [14]:
if __name__ == "__main__":
    # Sample user agent
    user_agent = "Weatherdata_collector/1.0" # you can name this whatever you want
    BASE_URL = "https://api.weather.gov"

    # Get forecast for a specific location - Durham
    lat, lon = 36.0, -79.0
    point_url = f"{BASE_URL}/points/{lat},{lon}"
    point_data = make_nws_request(point_url, user_agent)

    if point_data:
        # Step 2: Extract forecast URL
        forecast_url = point_data["properties"]["forecast"]

        # Step 3: Call forecast endpoint
        forecast_data = make_nws_request(forecast_url, user_agent)

        if forecast_data:
            # Step 4: Put into DataFrame
            periods = forecast_data["properties"]["periods"]

            df = pd.DataFrame(periods)[
                ["name", "startTime", "endTime", "isDaytime",
                 "temperature", "temperatureUnit", "windSpeed",
                 "windDirection", "shortForecast", "detailedForecast"]
            ]

            print("\n7-Day Forecast DataFrame:\n")
            print(df)
        else:
            print("Failed to retrieve forecast data.")
    else:
        print("Failed to retrieve point metadata.")


7-Day Forecast DataFrame:

               name                  startTime                    endTime  \
0    This Afternoon  2025-08-22T14:00:00-04:00  2025-08-22T18:00:00-04:00   
1           Tonight  2025-08-22T18:00:00-04:00  2025-08-23T06:00:00-04:00   
2          Saturday  2025-08-23T06:00:00-04:00  2025-08-23T18:00:00-04:00   
3    Saturday Night  2025-08-23T18:00:00-04:00  2025-08-24T06:00:00-04:00   
4            Sunday  2025-08-24T06:00:00-04:00  2025-08-24T18:00:00-04:00   
5      Sunday Night  2025-08-24T18:00:00-04:00  2025-08-25T06:00:00-04:00   
6            Monday  2025-08-25T06:00:00-04:00  2025-08-25T18:00:00-04:00   
7      Monday Night  2025-08-25T18:00:00-04:00  2025-08-26T06:00:00-04:00   
8           Tuesday  2025-08-26T06:00:00-04:00  2025-08-26T18:00:00-04:00   
9     Tuesday Night  2025-08-26T18:00:00-04:00  2025-08-27T06:00:00-04:00   
10        Wednesday  2025-08-27T06:00:00-04:00  2025-08-27T18:00:00-04:00   
11  Wednesday Night  2025-08-27T18:00:00-04:00  